<a href="https://colab.research.google.com/github/sydstewart/Orders/blob/main/Impsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""Companion code to https://realpython.com/simulation-with-simpy/
'Simulating Real-World Processes With SimPy'
Python version: 3.7.3
SimPy version: 3.0.11
"""
!pip install simpy
import simpy
import random
import statistics

wait_times = []


class Implement(object):
    def __init__(self, env, num_kitters):
        self.env = env
        self.kitter = simpy.Resource(env, num_kitters)
        # self.server = simpy.Resource(env, num_servers)
        # self.usher = simpy.Resource(env, num_ushers)

    def kit(self, project):
        yield self.env.timeout(random.randint(1, 3))

    # def check_ticket(self, project):
    #     yield self.env.timeout(3 / 60)

    # def sell_food(self, project):
    #     yield self.env.timeout(random.randint(1, 5))


def go_to_implement(env, project, implementation):
    # Moviegoer arrives at the implementation
    arrival_time = env.now

    with implementation.kitter.request() as request:
        yield request
        yield env.process(implementation.kit(project))

    # with implementation.usher.request() as request:
    #     yield request
    #     yield env.process(implementation.check_ticket(project))

    # if random.choice([True, False]):
    #     with implementation.server.request() as request:
    #         yield request
    #         yield env.process(implementation.sell_food(project))

    # Moviegoer heads into the implementation
    wait_times.append(env.now - arrival_time)


def run_implementation(env, num_kitters):
    implementation = Implement(env, num_kitters)

    for project in range(3):
        env.process(go_to_implement(env, project, implementation))

    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new person

        project += 1
        env.process(go_to_implement(env, project, implementation))


def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    # Pretty print the results
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)


# def get_user_input():
#     num_kitters = input("Input # of kitters working: ")
#     num_servers = input("Input # of servers working: ")
#     num_ushers = input("Input # of ushers working: ")
#     params = [num_kitters, num_servers, num_ushers]
#     if all(str(i).isdigit() for i in params):  # Check input is valid
#         params = [int(x) for x in params]
#     else:
#         print(
#             "Could not parse input. Simulation will use default values:",
#             "\n1 kitter, 1 server, 1 usher.",
#         )
#         params = [1, 1, 1]
#     return params


def main():
    # Setup
    random.seed(42)
    num_kitters = 4 #, num_servers, num_ushers = get_user_input()

    # Run the simulation
    env = simpy.Environment()
    env.process(run_implementation(env, num_kitters))
    env.run(until=90)

    # View the results
    mins, secs = get_average_wait_time(wait_times)
    print(
        "Running simulation...",
        f"\nThe average wait time is {mins} minutes and {secs} seconds.",
    )


if __name__ == "__main__":
    main()

Running simulation... 
The average wait time is 27 minutes and 15 seconds.
